In [1]:
from whisperx.vads import SileroCustom

default_vad_options = {
        "chunk_size": 30, # needed by silero since binarization happens before merge_chunks
        "vad_onset": 0.500,
        "vad_offset": 0.363,
        "vad_onnx": True,
        "silero_merge_cutoff": 0.1
    }

vad_model = SileroCustom(**default_vad_options)

>>Performing voice activity detection using Silero...


Using cache found in /Users/aayush/.cache/torch/hub/snakers4_silero-vad_master


In [2]:
import torch
import torchaudio
import os

audio_path = '/Users/aayush/nurix/DialogueManager/random/no_turn_ends/dataset_exp/no_turn_ends_audio/fe_03_03242_34.355_42.753.wav'
waveform, sample_rate = torchaudio.load(audio_path)
channel1 = waveform[0]  # Get the first channel
temp_path = 'temp.wav'
torchaudio.save(temp_path, channel1.unsqueeze(0), sample_rate)
print(f"Saved channel 1 to {temp_path}")

Saved channel 1 to temp.wav


In [3]:
from whisperx.audio import N_SAMPLES, SAMPLE_RATE, load_audio, log_mel_spectrogram
audio_path = 'temp.wav'
audio = load_audio(audio_path)

In [4]:
waveform = vad_model.preprocess_audio(audio)
z1 = {"waveform": waveform, "sample_rate": SAMPLE_RATE}

out1 = vad_model(z1)

In [5]:
vad_model.merge_chunks(out1, 30, 0.5, 0.363)

[{'start': 0.032, 'end': 2.176, 'segments': [(0.032, 2.176)]},
 {'start': 3.424, 'end': 4.0, 'segments': [(3.424, 4.0)]},
 {'start': 7.296, 'end': 8.416, 'segments': [(7.296, 8.416)]}]

In [22]:
audio

array([-4.8828125e-04, -3.0517578e-05,  4.8828125e-04, ...,
        1.4038086e-03, -2.4414062e-04, -1.1291504e-03], dtype=float32)

In [24]:
def data(audio, segments):
    for seg in segments:
        f1 = int(seg['start'] * SAMPLE_RATE)
        f2 = int(seg['end'] * SAMPLE_RATE)
        # print(f2-f1)
        yield {'inputs': audio[f1:f2]}
        
for i in data(audio, merged_segments):
    print(i)

{'inputs': array([ 0.00024414,  0.00027466,  0.00024414, ...,  0.00140381,
       -0.00024414, -0.00112915], dtype=float32)}


In [9]:
len(out)

5

In [23]:
voice_activity = predicts > 0.5

In [26]:

"""Convert voice activity tensor to time segments"""
segments = []
start_idx = None

# Find continuous voice segments
for i, is_voice in enumerate(voice_activity):
    if is_voice and start_idx is None:
        start_idx = i
    elif not is_voice and start_idx is not None:
        # Convert chunk indices to seconds (each chunk is 32ms)
        start_time = round(start_idx * 32 / 1000, 3)
        end_time = round(i * 32 / 1000, 3)
        
        # Store segment without applying max duration constraint
        segments.append((start_time, end_time))
        start_idx = None

# Handle case where voice activity continues until end
if start_idx is not None:
    start_time = round(start_idx * 32 / 1000, 3)
    end_time = round(len(voice_activity) * 32 / 1000, 3)
    segments.append((start_time, end_time))

In [27]:
segments

[(0.032, 1.504), (1.6, 2.176), (3.424, 4.0), (7.296, 7.712), (7.744, 8.416)]

In [19]:
vad_segments = vad_model({"waveform": waveform, "sample_rate": SAMPLE_RATE})

RuntimeError: Boolean value of Tensor with more than one value is ambiguous